In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!pip install gast==0.3.3
!pip install scipy 
!pip install Pillow 
!pip install cython 
!pip install matplotlib 
!pip install scikit-image>=0.14.2 
!pip install tensorflow==1.15.3 
!pip install keras==2.2.4 
!pip install opencv-python 
!pip install h5py 
!pip install imgaug
!pip install mrcnn

     |████████████████████████████████| 110.5MB 54kB/s 
     |████████████████████████████████| 512kB 45.9MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
     |████████████████████████████████| 3.8MB 48.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=96ee2c2d56320726769069c1f91a94a984f89b8161defa989258af7b28cae289
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninsta

In [15]:
import os
import tensorflow as tf
import mrcnn.config
import mrcnn.model
import mrcnn
import mrcnn.visualize
import cv2
import warnings
import urllib
warnings.filterwarnings('ignore')
import pandas as pd

print("Make sure it is 1.15.3 version before proceeding")
print(tf.__version__)

Make sure it is 1.15.3 version before proceeding
1.15.3


In [9]:
# parameters 

model_weigth = "/content/drive/Shareddrives/ComED Codes and Data/APIs/mask_rcnn_pole_cfg.h5"

image_dir = '/content/drive/Shareddrives/ComED Codes and Data/Dataset/Images/With UPC/'

outputfile = '/content/drive/Shareddrives/ComED Codes and Data/Final pole-detection/pole_data.csv'
metadata_file_location = '/content/drive/Shareddrives/ComED Codes and Data/Dataset/metadata.json'

In [10]:
# define the prediction configuration
class PredictionConfig(mrcnn.config.Config):
	# define the name of the configuration
	NAME = "pole_cfg"
	# number of classes (background + kangaroo)
	NUM_CLASSES = 1 + 1
	# simplify GPU config
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1

In [11]:
model = mrcnn.model.MaskRCNN(mode="inference", config=PredictionConfig(),model_dir=os.getcwd()) 

model.load_weights(filepath=model_weigth, by_name=True)








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.








In [13]:

data = {} 
final_data = [] 
size = '640x640'
API_key = 'Your_API_key'
heading =0
pitch =0
fov=120

def make_download_link(size,pano,heading,pitch,fov,key):
  data = {}
  data['size'] = size
  data['pano'] = pano
  data['heading'] = heading
  data['pitch'] = pitch
  data['fov'] = fov
  data['key'] = key
  url = 'https://maps.googleapis.com/maps/api/streetview?' + urllib.parse.urlencode(data)

  return url


for img in os.listdir(image_dir):
  image_location = os.path.join(image_dir, img)
  image = cv2.imread(image_location)
  pole_detected = model.detect([image], verbose=0)
  pole_detected = pole_detected[0]

  for x in range(len(pole_detected['scores'])):
    data['image_ID'] = img
    data['score'] = pole_detected['scores'][x]
    bbox = pole_detected['rois'][x]
    data['Y Top'] = bbox[0]
    data['X min'] = bbox[1]
    data['Y bottom'] =bbox[2]
    data['X max'] = bbox[3]

    data['Width'] = abs(bbox[3] - bbox[1])
    data['Height'] = abs(bbox[2] - bbox[0])

    final_data.append(data.copy())


pole_location_data = pd.DataFrame.from_dict(final_data)
metadata = pd.read_json(metadata_file_location)

pole_data = pd.merge(
    pole_location_data,
    metadata,
    how="inner",
    left_on='image_ID',
    right_on='_file',
    left_index=False,
    right_index=False)

NameError: ignored

In [18]:
pole_data.head()

,image_ID,score,Y Top,X min,Y bottom,X max,Width,Height,copyright,date,location,pano_id,status,_file
0,gsv_152.jpg,0.921575,223,495,339,519,24,116,© Google,2019-07-01,"{'lat': 41.92678768831036, 'lng': -87.68170650...",u_LDItdWHTEs7hpyPBC9Mg,OK,gsv_152.jpg
1,gsv_152.jpg,0.807345,65,45,400,112,67,335,© Google,2019-07-01,"{'lat': 41.92678768831036, 'lng': -87.68170650...",u_LDItdWHTEs7hpyPBC9Mg,OK,gsv_152.jpg
2,gsv_153.jpg,0.980155,51,159,386,206,47,335,© Google,2019-07-01,"{'lat': 41.92678768831036, 'lng': -87.68170650...",u_LDItdWHTEs7hpyPBC9Mg,OK,gsv_153.jpg
3,gsv_157.jpg,0.957695,185,580,342,631,51,157,© Google,2019-07-01,"{'lat': 41.92678768831036, 'lng': -87.68170650...",u_LDItdWHTEs7hpyPBC9Mg,OK,gsv_157.jpg
4,gsv_157.jpg,0.789528,292,573,363,591,18,71,© Google,2019-07-01,"{'lat': 41.92678768831036, 'lng': -87.68170650...",u_LDItdWHTEs7hpyPBC9Mg,OK,gsv_157.jpg


In [20]:
pole_data['url'] = pole_data['pano_id'].apply(lambda x: make_download_link(size,x,heading,pitch,fov,API_key))


,image_ID,score,Y Top,X min,Y bottom,X max,Width,Height,copyright,date,location,pano_id,status,_file,url
0,gsv_152.jpg,0.921575,223,495,339,519,24,116,© Google,2019-07-01,"{'lat': 41.92678768831036, 'lng': -87.68170650...",u_LDItdWHTEs7hpyPBC9Mg,OK,gsv_152.jpg,https://maps.googleapis.com/maps/api/streetvie...
1,gsv_152.jpg,0.807345,65,45,400,112,67,335,© Google,2019-07-01,"{'lat': 41.92678768831036, 'lng': -87.68170650...",u_LDItdWHTEs7hpyPBC9Mg,OK,gsv_152.jpg,https://maps.googleapis.com/maps/api/streetvie...
2,gsv_153.jpg,0.980155,51,159,386,206,47,335,© Google,2019-07-01,"{'lat': 41.92678768831036, 'lng': -87.68170650...",u_LDItdWHTEs7hpyPBC9Mg,OK,gsv_153.jpg,https://maps.googleapis.com/maps/api/streetvie...
3,gsv_157.jpg,0.957695,185,580,342,631,51,157,© Google,2019-07-01,"{'lat': 41.92678768831036, 'lng': -87.68170650...",u_LDItdWHTEs7hpyPBC9Mg,OK,gsv_157.jpg,https://maps.googleapis.com/maps/api/streetvie...
4,gsv_157.jpg,0.789528,292,573,363,591,18,71,© Google,2019-07-01,"{'lat': 41.92678768831036, 'lng': -87.68170650...",u_LDItdWHTEs7hpyPBC9Mg,OK,gsv_157.jpg,https://maps.googleapis.com/maps/api/streetvie...


In [21]:
pole_data['url'] = pole_data['pano_id'].apply(lambda x: make_download_link(size,x,heading,pitch,fov,API_key))
pole_data.drop('_file', axis=1).to_csv(outputfile, encoding='utf-8', index=False)